이전보다 더 깊은 CNN 모델을 구현한 코드이다.

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda': torch.cuda.manual_seed_all(777)

# set parmeters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# define datasets
mnist_train = dsets.MNIST(root='MNIST_Data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_Data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=len(mnist_test), shuffle=False)

# design model
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.keep_prob = 0.5
    # First Layer (Convolution Layer)
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # Second Layer (Convolution Layer)
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # Third Layer (Convolution Layer)
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )
    # Forth Layer (Fully Connected Layer)
    self.fc1 = nn.Linear(4*4*128, 625, bias=True)
    nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p=1-self.keep_prob)
    )

    # Fifth Layer (Fully Connected Layer)
    self.layer5 = nn.Linear(625, 10, bias=True)
    nn.init.xavier_uniform_(self.layer5.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.layer5(out)
    return out

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
total_batch = len(test_loader)
for epoch in range(training_epochs):
  for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)

    hypothesis = model(x)
    cost = criterion(hypothesis, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost = cost / total_batch
  
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.0814922601
[Epoch:    2] cost = 0.0144035034
[Epoch:    3] cost = 0.00762545038
[Epoch:    4] cost = 0.0102440119
[Epoch:    5] cost = 0.0334713012
[Epoch:    6] cost = 0.0494290404
[Epoch:    7] cost = 0.024852464
[Epoch:    8] cost = 0.0386896692
[Epoch:    9] cost = 0.0490666963
[Epoch:   10] cost = 0.00230287458
[Epoch:   11] cost = 0.00892378483
[Epoch:   12] cost = 0.000447678787
[Epoch:   13] cost = 0.000663999934
[Epoch:   14] cost = 0.000416760566
[Epoch:   15] cost = 0.0643055737


In [11]:
model.eval()

with torch.no_grad():
  for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)
    
    prediction = model(x)
    correct_prediction = torch.argmax(prediction, 1) == y
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9932999610900879


출처: https://wikidocs.net/63618